[![Binder](https://mybinder.org/badge_logo.svg)](https://github.com/Sistemas-Multimedia/MRVC/blob/master/src/image_IPP.ipynb)

# RD performance in IPP video coding

This notebook runs different IPP... experiments. The first one, implemented in `image_IPP.py` is a simple IPP... video codec without macro-blocks (MBs) MB-type RDO. All MBs are motion compensated as P-type blocks.  The second one, implemented in `image_IPP_adaptive.py`, is identical to `image_IPP.py`, but MB-type RDO is used (depending on the RD slope of the differerent MB types). S/P/I-type MBs are used. Finally, the RD curves are compared to MP4, using intra-mode.

In [1]:
!ln -sf ~/quantization/deadzone_quantizer.py .
!ln -sf ~/quantization/distortion.py .
!ln -sf ~/quantization/information.py .
!ln -sf ~/quantization/midtread_quantizer.py .

### Prepare the sequence

In [2]:
!rm /tmp/*.png

In [3]:
#sequence = "bus"
sequence = "akiyo_cif.y4m"

In [4]:
N_frames = 5 # Min val 3

In [5]:
!../sequences/extract_frames.sh -s $sequence -n $N_frames

sequence file name = akiyo_cif.y4m
number of frames to extract = 5
/home/vruiz/MRVC/sequences/akiyo_cif.y4m exists. Only extracting ...
ffmpeg version n4.4.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11.1.0 (GCC)
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-amf --enable-avisynth --enable-cuda-llvm --enable-lto --enable-fontconfig --enable-gmp --enable-gnutls --enable-gpl --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libdrm --enable-libfreetype --enable-libfribidi --enable-libgsm --enable-libiec61883 --enable-libjack --enable-libmfx --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopus --enable-libpulse --enable-librav1e --enable-librsvg --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtheora --enable-libv4l2 --enable-libvidstab --enable-libvmaf -

## Testing `image_IPP`

### Select `image_IPP` codec in `config.py`
All MBs in P-frames are P-MBs.

In [6]:
!sed -z -i 's/\n#temporal_codec = "image_IPP"\n/\ntemporal_codec = "image_IPP"\n/g' config.py
!sed -z -i 's/\ntemporal_codec = "image_IPP_adaptive"\n/\n#temporal_codec = "image_IPP_adaptive"\n/g' config.py
!sed -z -i 's/\ntemporal_codec = "MP4"\n/\n#temporal_codec = "MP4"\n/g' config.py

In [7]:
!cat -n config.py | grep "temporal_codec" 

    27	temporal_codec = "image_IPP"
    28	#temporal_codec = "image_IPP_adaptive"
    29	#temporal_codec = "MP4"


### Select the spatial+entropy codec
We use MP4 and therefore, no previous spatial transform is used. MP4 uses the DCT.

In [8]:
!sed -z -i 's/\ntransform = "DWT"\n/\n#transform = "DWT"\n/g' config.py
!sed -z -i 's/\ntransform = "LP"\n/\n#transform = "LP"\n/g' config.py
!sed -z -i 's/\n#transform = None\n/\ntransform = None\n/g' config.py

In [9]:
!cat -n config.py | grep "transform" 

     6	#multiresolution_transform = "DWT"
     7	#multiresolution_transform = "LP"
     8	multiresolution_transform = None


### Select the color transform

In [10]:
!sed -z -i 's/\ncolor = "YCoCg"\n/\n#color = "YCoCg"\n/g' config.py
!sed -z -i 's/\ncolor = "YCoCg"\n/\n#color = "YCoCg"\n/g' config.py
!sed -z -i 's/\n#color = "YCoCg"\n/\ncolor = "YCoCg"\n/g' config.py

In [11]:
!cat -n config.py | grep "color" 

    14	color = "YCoCg"
    15	#color = "YCrCb"
    16	#color = "RGB"


### Select the spatial codec

In [12]:
!sed -z -i 's/\nspatial_codec = "Q+PNG"\n/\n#spatial_codec = "Q+PNG"\n/g' config.py
!sed -z -i 's/\nspatial_codec = "H264"\n/\n#spatial_codec = "H264"\n/g' config.py
!sed -z -i 's/\n#spatial_codec = "DCT"\n/\nspatial_codec = "DCT"\n/g' config.py

In [13]:
!cat -n config.py | grep "spatial_codec" 

    10	#spatial_codec = "Q+PNG"
    11	#spatial_codec = "H264"
    12	spatial_codec = "DCT"


### Select quantization range

In [14]:
!sed -z -i 's/\nQ_steps = range(42, 21, -3)\n/\n#Q_steps = range(42, 21, -3)\n/g' config.py
!sed -z -i 's/\n#Q_steps = \[128, 64, 32, 16, 8\]\n/\nQ_steps = \[128, 64, 32, 16, 8\]\n/g' config.py

In [15]:
!cat -n config.py | grep "Q_steps" 

    22	#Q_steps = range(42, 21, -3)
    25	Q_steps = [128, 64, 32, 16, 8]


### Run the codec

!rm /tmp/*.png
!../sequences/$sequence/runme.sh

In [ ]:
# You can see in real-time the output by running "tail -f /tmp/output"
!python RD_curve.py --first_frame=0 --N_frames=$N_frames 1> >(tee /tmp/output_image_IPP >&1) | grep BPP | grep Average | cat -n

[image_IPP.py:56 encode() INFO] Running ...
[image_3.py:51 _write() DEBUG] optipng /tmp/original_texture_000.png
** Processing: /tmp/original_texture_000.png
352x288 pixels, 3x8 bits/pixel, RGB
Reducing image to 8 bits/pixel, 153 colors in palette
Input IDAT size = 2822 bytes
Input file size = 2879 bytes

Trying:
  zc = 9  zm = 8  zs = 0  f = 0		IDAT size = 1708
                               
Selecting parameters:
  zc = 9  zm = 8  zs = 0  f = 0		IDAT size = 1708

Output IDAT size = 1708 bytes (1114 bytes decrease)
Output file size = 2236 bytes (643 bytes = 22.33% decrease)

[image_3.py:51 _write() DEBUG] optipng /tmp/original_reconstructed_000.png
** Processing: /tmp/original_reconstructed_000.png
352x288 pixels, 3x8 bits/pixel, RGB
Input IDAT size = 25459 bytes
Input file size = 25552 bytes

Trying:
  zc = 9  zm = 8  zs = 0  f = 5		IDAT size = 25459
  zc = 9  zm = 8  zs = 1  f = 5		IDAT size = 25371
                               
Selecting parameters:
  zc = 9  zm = 8  zs = 1  f = 

                               
Selecting parameters:
  zc = 9  zm = 8  zs = 0  f = 0		IDAT size = 1228

Output IDAT size = 1228 bytes (909 bytes decrease)
Output file size = 1285 bytes (909 bytes = 41.43% decrease)

[image_3.py:51 _write() DEBUG] optipng /tmp/original_reconstructed_002.png
** Processing: /tmp/original_reconstructed_002.png
352x288 pixels, 3x8 bits/pixel, RGB
Input IDAT size = 25679 bytes
Input file size = 25772 bytes

Trying:
  zc = 9  zm = 8  zs = 0  f = 5		IDAT size = 25679
  zc = 9  zm = 8  zs = 1  f = 5		IDAT size = 25603
                               
Selecting parameters:
  zc = 9  zm = 8  zs = 1  f = 5		IDAT size = 25603

Output IDAT size = 25603 bytes (76 bytes decrease)
Output file size = 25660 bytes (112 bytes = 0.43% decrease)

[image_IPP.py:73 encode() DEBUG] W_k 32 -50
[image_IPP.py:75 encode() DEBUG] COMPUTED flow 0.2449968159198761 -0.286998450756073
[image_1.py:44 _write() DEBUG] optipng /tmp/original_motion_y_003.png
** Processing: /tmp/original_moti

  zc = 9  zm = 8  zs = 0  f = 5		IDAT size = 25864
  zc = 9  zm = 8  zs = 1  f = 5		IDAT size = 25791
                               
Selecting parameters:
  zc = 9  zm = 8  zs = 1  f = 5		IDAT size = 25791

Output IDAT size = 25791 bytes (73 bytes decrease)
Output file size = 25848 bytes (109 bytes = 0.42% decrease)

[image_IPP.py:118 compute_br_DCT() INFO] height=288 width=352 n_channels=3 sequence_time=0.16666666666666666
[image_IPP.py:127 compute_br_DCT() DEBUG] cat /tmp/original_texture_???.png | gzip -9 > /tmp/image_IPP_texture.gz
[image_IPP.py:133 compute_br_DCT() INFO] texture: 2605 bytes, 125.04 KBPS, 0.013704755892255891 BPP
[image_1.py:44 _write() DEBUG] optipng /tmp/original_motion_y_diff_comp_002.png
** Processing: /tmp/original_motion_y_diff_comp_002.png
22x18 pixels, 16 bits/pixel, grayscale
Input IDAT size = 24 bytes
Input file size = 81 bytes

Trying:
  zc = 9  zm = 8  zs = 0  f = 0		IDAT size = 21
  zc = 9  zm = 8  zs = 1  f = 0		IDAT size = 21
                       

[image_3.py:51 _write() DEBUG] optipng /tmp/original_dequantized_prediction_error_001.png
** Processing: /tmp/original_dequantized_prediction_error_001.png
352x288 pixels, 3x8 bits/pixel, RGB
Input IDAT size = 6581 bytes
Input file size = 6638 bytes

Trying:
  zc = 9  zm = 8  zs = 0  f = 5		IDAT size = 6581
                               
/tmp/original_dequantized_prediction_error_001.png is already optimized.

[image_3.py:51 _write() DEBUG] optipng /tmp/original_reconstructed_001.png
** Processing: /tmp/original_reconstructed_001.png
352x288 pixels, 3x8 bits/pixel, RGB
Input IDAT size = 43240 bytes
Input file size = 43357 bytes

Trying:
  zc = 9  zm = 8  zs = 0  f = 5		IDAT size = 43240
  zc = 9  zm = 8  zs = 1  f = 5		IDAT size = 43052
                               
Selecting parameters:
  zc = 9  zm = 8  zs = 1  f = 5		IDAT size = 43052

Output IDAT size = 43052 bytes (188 bytes decrease)
Output file size = 43109 bytes (248 bytes = 0.57% decrease)

[image_IPP.py:73 encode() DEBUG] 

In [ ]:
!grep BPP /tmp/output_image_IPP | grep Average | cut -d ' ' -f 4,9 | tr ' ' '\t'  > /tmp/image_IPP.txt

In [ ]:
!cat /tmp/image_IPP.txt

In [ ]:
RD_image_IPP = []
with open("/tmp/image_IPP.txt", 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        RD_image_IPP.append((float(rate), float(_distortion)))

## Testing `image_IPP_adaptive.py`
Now I-type MBs are allowed in P-frames.

### Select `image_IPP_adaptive` codec in `config.py`

In [ ]:
!sed -z -i 's/\ntemporal_codec = "image_IPP"\n/\n#temporal_codec = "image_IPP"\n/g' config.py
!sed -z -i 's/\n#temporal_codec = "image_IPP_adaptive"\n/\ntemporal_codec = "image_IPP_adaptive"\n/g' config.py
!sed -z -i 's/\ntemporal_codec = "MP4"\n/\n#temporal_codec = "MP4"\n/g' config.py

In [ ]:
!cat -n config.py | grep "temporal_codec" 

In [ ]:
!sed -z -i 's/\ntransform = "DWT"\n/\n#transform = "DWT"\n/g' config.py
!sed -z -i 's/\ntransform = "LP"\n/\n#transform = "LP"\n/g' config.py
!sed -z -i 's/\n#transform = None\n/\ntransform = None\n/g' config.py

In [ ]:
!cat -n config.py | grep "transform" 

In [ ]:
!sed -z -i 's/\ncolor = "YCoCg\n/\n#color = "YCoCg\n/g' config.py
!sed -z -i 's/\ncolor = "YCoCg\n/\n#color = "YCoCg\n/g' config.py
!sed -z -i 's/\n#color = "YCoCg\n/\ncolor = "YCoCg\n/g' config.py

In [ ]:
!cat -n config.py | grep "color" 

In [ ]:
!sed -z -i 's/\nQ_steps = range(42, 21, -3)\n/\n#Q_steps = range(42, 21, -3)\n/g' config.py
!sed -z -i 's/\n#Q_steps = \[128, 64, 32, 16, 8\]\n/\nQ_steps = \[128, 64, 32, 16, 8\]\n/g' config.py

In [ ]:
!cat -n config.py | grep "Q_steps" 

!rm /tmp/*.png
!../sequences/$sequence/runme.sh

In [ ]:
# You can see in real-time the output by running "tail -f /tmp/output"
!python RD_curve.py --first_frame=0 --N_frames=$N_frames 1> >(tee /tmp/output_image_IPP_adaptive >&1) | grep BPP | grep Average | cat -n

In [ ]:
!grep BPP /tmp/output_image_IPP_adaptive | grep Average | cut -d ' ' -f 4,9 | tr ' ' '\t'  > /tmp/image_IPP_adaptive.txt

In [ ]:
!cat /tmp/image_IPP_adaptive.txt

In [ ]:
RD_image_IPP_adaptive = []
with open("/tmp/image_IPP_adaptive.txt", 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        RD_image_IPP_adaptive.append((float(rate), float(_distortion)))

## Testing MP4

### Select `MP4` codec in `config.py`

In [ ]:
!sed -z -i 's/\ntemporal_codec = "image_IPP"\n/\n#temporal_codec = "image_IPP"\n/g' config.py
!sed -z -i 's/\ntemporal_codec = "image_IPP_adaptive"\n/\n#temporal_codec = "image_IPP_adaptive"\n/g' config.py
!sed -z -i 's/\n#temporal_codec = "MP4"\n/\ntemporal_codec = "MP4"\n/g' config.py

In [ ]:
!cat -n config.py | grep "temporal_codec" 

In [ ]:
!sed -z -i 's/\ntransform = "DWT"\n/\n#transform = "DWT"\n/g' config.py
!sed -z -i 's/\ntransform = "LP"\n/\n#transform = "LP"\n/g' config.py
!sed -z -i 's/\n#transform = None\n/\ntransform = None\n/g' config.py

In [ ]:
!cat -n config.py | grep "transform" 

In [ ]:
!sed -z -i 's/\ncolor = "YCoCg\n/\n#color = "YCoCg\n/g' config.py
!sed -z -i 's/\ncolor = "YCoCg\n/\n#color = "YCoCg\n/g' config.py
!sed -z -i 's/\n#color = "YCoCg\n/\ncolor = "YCoCg\n/g' config.py

In [ ]:
!cat -n config.py | grep "color" 

In [ ]:
!sed -z -i 's/\n#Q_steps = range(42, 21, -3)\n/\nQ_steps = range(42, 21, -3)\n/g' config.py
!sed -z -i 's/\nQ_steps = \[128, 64, 32, 16, 8\]\n/\n#Q_steps = \[128, 64, 32, 16, 8\]\n/g' config.py

In [ ]:
!cat -n config.py | grep "Q_steps" 

!rm /tmp/*.png
!../sequences/$sequence/runme.sh

In [ ]:
!python RD_curve.py --N_frames=$N_frames 1> >(tee /tmp/output_MP4 >&1) | grep BPP | grep Average | cat -n

In [ ]:
!grep BPP /tmp/output_MP4 | grep Average | cut -d ' ' -f 4,9 | tr ' ' '\t'  > /tmp/MP4.txt

In [ ]:
!cat /tmp/MP4.txt

In [ ]:
RD_MP4 = []
with open("/tmp/MP4.txt", 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        RD_MP4.append((float(rate), float(_distortion)))

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import pylab

pylab.figure(dpi=150)
pylab.plot(*zip(*RD_image_IPP), label="image_IPP")
pylab.plot(*zip(*RD_image_IPP_adaptive), label="image_IPP_adaptive")
#pylab.plot(*zip(*RD_image_IPP_QR), label="image_IPP_quantized_residue")
pylab.plot(*zip(*RD_MP4), label="MP4")
pylab.title(f"{sequence}")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("AMSE")
plt.legend(loc='upper right')
pylab.show()

## Ignore the rest

## Testing `image_IPP_quantized_residue`

`image_IPP_quantized_residue.py` is a modification of `image_IPP.py` where the prediction-error images are quantized before they are compressed with MP4. Each macro-block is quantized to contribute with the same slope to the reconstructed macro-block. 


### Select `image_IPP_quantized_residue` in `RD_curve.py`

!sed -z -i 's/\nimport image_IPP as codec\n/\n#import image_IPP as codec\n/g' RD_curve.py
!sed -z -i 's/\nimport image_IPP_adaptive as codec\n/\n#import image_IPP_adaptive as codec\n/g' RD_curve.py
!sed -z -i 's/\nimport MP4 as codec\n/\n#import MP4 as codec\n/g' RD_curve.py
!sed -z -i 's/\nimport IPP_compressor as codec\n/\n#import IPP_compressor as codec\n/g' RD_curve.py
!sed -z -i 's/\nimage_IPP_quantized_prediction as codec\n/\n#import image_IPP_quantized_prediction as codec\n/g' RD_curve.py
!sed -z -i 's/\n#import image_IPP_quantized_residue as codec\n/\nimport image_IPP_quantized_residue as codec\n/g' RD_curve.py

In [ ]:
!cat -n RD_curve.py | grep "import" 

!sed -z -i 's/\ntransform = "DWT"\n/\n#transform = "DWT"\n/g' config.py
!sed -z -i 's/\ntransform = "LP"\n/\n#transform = "LP"\n/g' config.py
!sed -z -i 's/\n#transform = None\n/\ntransform = None\n/g' config.py

In [ ]:
!cat -n config.py | grep "transform" 

!sed -z -i 's/\ncolor = "YCoCg\n/\n#color = "YCoCg\n/g' config.py
!sed -z -i 's/\ncolor = "YCoCg\n/\n#color = "YCoCg\n/g' config.py
!sed -z -i 's/\n#color = "YCoCg\n/\ncolor = "YCoCg\n/g' config.py

In [ ]:
!cat -n config.py | grep "color" 

In [ ]:
# You can see in real-time the output by running "tail -f /tmp/output"
#!python RD_curve.py --first_frame=0 --N_frames=$N_frames 1> >(tee /tmp/output_image_IPP_quantized_residue >&1) | grep BPP | grep Average | cat -n

In [ ]:
!grep BPP /tmp/output_image_IPP_quantized_residue | grep Average | cut -d ' ' -f 4,9 | tr ' ' '\t'  > /tmp/image_IPP_quantized_residue.txt

In [ ]:
!cat /tmp/image_IPP_quantized_residue.txt

In [ ]:
RD_image_IPP_QR = []
with open("/tmp/image_IPP_quantized_residue.txt", 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        RD_image_IPP_QR.append((float(rate), float(_distortion)))

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import pylab

pylab.figure(dpi=150)
pylab.plot(*zip(*RD_image_IPP), label="image_IPP")
pylab.plot(*zip(*RD_image_IPP_adaptive), label="image_IPP_adaptive")
pylab.plot(*zip(*RD_image_IPP_QR), label="image_IPP_quantized_residue")
pylab.plot(*zip(*RD_MP4), label="MP4")
pylab.title(f"{sequence}")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("AMSE")
plt.legend(loc='upper right')
pylab.show()

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import pylab

pylab.figure(dpi=150)
pylab.plot(*zip(*RD_points_no_intrablocks), label="No Intra-blocks")
pylab.plot(*zip(*RD_points_with_intrablocks), label="With Intra-blocks")
pylab.plot(*zip(*RD_points_MP4), label="MP4")
pylab.title(f"{sequence}")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("AMSE")
plt.legend(loc='upper right')
pylab.show()